# Imports

In [1]:
import numpy as np
import pandas as pd
import scipy

# Introduction

First of all, we will use artificial small dataset to ease understanding of the general operations.

Let us consider a matrix $[X]_{N,p}$ with $N=7$ observations and $p=3$ features.

We will cluster into $c=2$ clusters.

In [2]:
n_samples = 7
c = 2

with a priori knowledge about first 3 observations included in the matrix $F$:
<br>
$$
F=
\begin{bmatrix}
1 & 0 \\
1 & 0 \\
0 & 1 \\
0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\
\end{bmatrix}
$$

In [3]:
F = np.array(
[[1, 0], [1, 0], [0, 1], [0, 0], [0, 0], [0, 0], [0, 0]]    
)

We need to set the **scaling factor** $\alpha$, the most important parameter of SSFCM.

In [4]:
alpha = 0.8

Matrix $X$ will be simulated by sampling from $\mathcal{N}(\mu_p, 3)$, $\mu_p = 5, 10, 15$.

In [5]:
rng = np.random.default_rng(12345)

In [6]:
X = np.hstack((
    rng.normal(loc=5, scale=3, size=7).reshape(n_samples, 1),
    rng.normal(loc=10, scale=3, size=7).reshape(n_samples, 1),
    rng.normal(loc=15, scale=3, size=7).reshape(n_samples, 1)
))

In [7]:
pd.DataFrame(X, columns=["X1", "X2", "X3"]).to_csv("X.csv", index=False)

# The optimization procedure

The optimiziation procedure in SSFCM is an iterative procedure.

We optimize two variables: matrix $U$ and matrix $V$ until a selected convergence criterion is met.

Instead of optimizing it at the same time, which is unfeasible, we keep one of them fixed (e.g. $U$), and use it as the input when optimizing the other (in this example: $V$).

Such a procedure requires something to start with - thus we simulate a random matrix $U$ and normalize it so it satisfies the conditions of the optimization problem in step $l=0$, and then in each step $l=1, \ldots$ we first:<br>
1. keep $U$ fixed and calculate $V$ (since $U$ together with $X$ is an input to $V$),
2. keep the $V$ calculated in the step above fixed and calculate updated $U$ (since $V$ together with $X$ is an input to $U$).

# Step $l=0$ initiate dummy $U$

In [8]:
u_not_normalized = rng.uniform(size=(n_samples, c))

In [9]:
u_not_normalized

array([[0.1598956 , 0.34010018],
       [0.46519315, 0.26642103],
       [0.8157764 , 0.19329439],
       [0.12946908, 0.09166475],
       [0.59856801, 0.8547419 ],
       [0.60162124, 0.93198836],
       [0.72478136, 0.86055132]])

We need to assure the condition $\sum_{k=1}^2 u_{jk} = 1 $ is met.

To this end, we calculate row-wise sums, and present it in a form of a $\text{n_samples} \times 1$ matrix.

In [10]:
row_sums = u_not_normalized.sum(axis=1)[np.newaxis].T

In [11]:
row_sums

array([[0.49999579],
       [0.73161418],
       [1.00907079],
       [0.22113383],
       [1.45330992],
       [1.5336096 ],
       [1.58533268]])

We can now use `np.tile(A, reps)` function [(see doc)](https://numpy.org/doc/stable/reference/generated/numpy.tile.html)
to "Construct an array by repeating A the number of times given by reps".

In [12]:
np.tile(row_sums, c)

array([[0.49999579, 0.49999579],
       [0.73161418, 0.73161418],
       [1.00907079, 1.00907079],
       [0.22113383, 0.22113383],
       [1.45330992, 1.45330992],
       [1.5336096 , 1.5336096 ],
       [1.58533268, 1.58533268]])

Finally, we arrive at a normalized matrix $U^{(l=0)}$

In [13]:
U_dummy = u_not_normalized / row_sums

In [14]:
U_dummy

array([[0.3197939 , 0.6802061 ],
       [0.63584491, 0.36415509],
       [0.80844318, 0.19155682],
       [0.58547838, 0.41452162],
       [0.41186536, 0.58813464],
       [0.392291  , 0.607709  ],
       [0.45717935, 0.54282065]])

A simple check shows us `U` satisfies the requested condition:

In [15]:
U_dummy.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1.])

Initiliaze `U` as the first matrix with values from `U_dummy`

In [16]:
U = U_dummy

# Step $l=1$ calculate prototypes matrix $V$

The regular FCM algorithm provides a formula for each $v_k$ on its own:

$$
v_k = \frac{
\sum\limits_{j=1}^{7} u^2_{jk} \cdot \mathbf{x}_j
}
{
\sum\limits_{j=1}^{7} u^2_{jk}
}
$$

Using `numpy`, one can achieve by multiplying the transposed $X$ and $U$, and transposing the result of this multiplication.

In [17]:
(X.T @ U**2 / np.sum(U**2, axis=0)).T

array([[ 3.9605454 ,  9.96556552, 15.88707116],
       [ 2.89866951, 11.65441908, 16.14858076]])

<p span style="color:red">
PLACEHOLDER FOR SHOWING THAT THIS MATRIX CALCULATION IS OK    
</p>

In case of SSFCM, the formula is
$$
v_k = \frac{
\sum\limits_{j=1}^7 \phi_{jk} \cdot \mathbf{x}_j
}{
\sum\limits_{j=1}^7 \phi_{jk}
}
$$
where
$$
\phi_{jk} = 
u^2_{jk} + \alpha b_j (u_{jk} - f_{jk})^2.
$$

So, we simply need to calculate matrix $\Phi = [\phi_{jk}]$, and perform the same operation as above.

In [18]:
U

array([[0.3197939 , 0.6802061 ],
       [0.63584491, 0.36415509],
       [0.80844318, 0.19155682],
       [0.58547838, 0.41452162],
       [0.41186536, 0.58813464],
       [0.392291  , 0.607709  ],
       [0.45717935, 0.54282065]])

In [19]:
F

array([[1, 0],
       [1, 0],
       [0, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])

In [20]:
UF = alpha * (U - F)**2

In [21]:
UF

array([[0.37014427, 0.37014427],
       [0.10608714, 0.10608714],
       [0.5228643 , 0.5228643 ],
       [0.27422795, 0.13746254],
       [0.13570646, 0.27672188],
       [0.12311378, 0.29544818],
       [0.16721037, 0.23572341]])

`UF` corresponds to the $\alpha b_j (u_{jk} - f_{jk})^2$, but we still need to include $b_j$, i.e. zero out this term for non-supervised observations indexed by $h$.

In a matrix form, we can do it by setting the indices, and updating corresponding rows of `UF` setting them to $0$.

In [22]:
i_indices = F.sum(axis=1).nonzero()[0]

In [23]:
i_indices

array([0, 1, 2])

In [24]:
j_indices = np.arange(0, F.shape[0])

In [25]:
j_indices

array([0, 1, 2, 3, 4, 5, 6])

In [26]:
h_indices = np.setdiff1d(j_indices, i_indices)

In [27]:
h_indices

array([3, 4, 5, 6])

In [28]:
UF[h_indices] = 0.

In [29]:
UF

array([[0.37014427, 0.37014427],
       [0.10608714, 0.10608714],
       [0.5228643 , 0.5228643 ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ]])

The final step to obtain $\Phi$ is

In [30]:
Phi = U**2 + UF

In [31]:
Phi

array([[0.47241241, 0.83282462],
       [0.51038589, 0.23869607],
       [1.17644468, 0.55955832],
       [0.34278494, 0.17182817],
       [0.16963308, 0.34590235],
       [0.15389223, 0.36931023],
       [0.20901296, 0.29465426]])

You can see that `Phi` ($\Phi$) is different from `U**2` only for the supervised rows indexed by $i$

In [32]:
U**2

array([[0.10226814, 0.46268034],
       [0.40429875, 0.13260893],
       [0.65358038, 0.03669401],
       [0.34278494, 0.17182817],
       [0.16963308, 0.34590235],
       [0.15389223, 0.36931023],
       [0.20901296, 0.29465426]])

Then, matrix $V$ in SSFCM is calculated by

Previous code from FCM that we need to reflect using `Phi`
```python
(X.T @ U**2 / np.sum(U**2, axis=0)).T
```

In [33]:
V = (X.T @ Phi / np.sum(Phi, axis=0)).T

In [34]:
V

array([[ 3.4642421 ,  9.24276663, 15.82553729],
       [ 2.74040953, 10.27474641, 15.98930677]])

# Step $l=1$ calculate memberships matrix $U$

In the classical, unsupervised case, we only deal with memberships $u_{jk}$ of $j-$th observation to $k-$th cluster.

With the semi-supervised case, we have 3 types of memberships:
1. membership of an unsupervised observation $h$ to any $k-$th cluster: $u_{hk}$,
2. membership of a supervised observation $i$ to any non-supervised cluster $k \neq s(i)$: $u_{i,k \neq s(i)}$,
3. membership of a supervised observation $i$ to a supervised cluster $s(i)$: $u_{i,s(i)}$.

A general formula for $u_{jk}$ in case of SSFCM is

$$
\begin{equation}
\begin{aligned}
\hat{u}_{jk} = {} 
    \frac{
        1 + \alpha
        \cdot 
        \big( 1 - b_j \cdot \sum_{s=1}^{c} f_{js} \big)
    }{
        1 + \alpha
    }  
    \cdot
        \frac{
            1
        }{
            \sum_{s=1}^{c} 
            \big( {d_{jk}^2} / {d_{js}^2} \big)
        }
    +
    \frac{\alpha}{1+\alpha}
    \cdot 
    \big(
    f_{jk} \cdot b_j
    \big)
    .
\end{aligned}
\label{ujk-formula-simple}
\end{equation}
$$

## Evidence matrix $E$

One can notice that the part 
$$
\frac{
1
}{
\sum_{s=1}^{c} 
\big( {d_{jk}^2} / {d_{js}^2} \big)
}
=
{
\bigg(
    \sum_{s=1}^{c}
    \frac{
        d^2(x_j, v_k)
    }{
       d^2(x_j, v_s)
    }
\bigg)
}^{-1}
$$

that we call **data evidence** is common for all 3 distinc types of memberships in SSFCM.

The **evidence** matrix $E = [e_{jk}]$ will be our main building block, where 
$$
e_{jk} = {
\bigg(
    \sum_{s=1}^{c}
    \frac{
        d^2(x_j, v_k)
    }{
       d^2(x_j, v_s)
    }
\bigg)
}^{-1}
$$

Evidence matrix $E$ will be built from the distances matrix $D = [d_{jk}]$.

### `Python` code

To compute distances between each point $x_j \in R^3$ and each of $v_1, v_2$, we will use a `cdist` function from `scipy` [(docs)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html)

In [35]:
cdist = scipy.spatial.distance.cdist

In [36]:
D = cdist(X, V, metric='Euclidean')**2

In [37]:
D

array([[19.75213031, 12.55593133],
       [32.77869472, 38.63756415],
       [29.55676429, 39.6387398 ],
       [82.52554128, 70.64639669],
       [15.94460273, 11.60122044],
       [14.1513846 , 16.80759488],
       [28.45890242, 18.17946634]])

Each element $d_{y,z}$ from $D$ is a distance between $y-$th element from $X$ ($y-$th observation) and $z-$th element from $V$ ($z-$th cluster's prototype)

It turns out the same operation can be carried out using pure `numpy`, namely `numpy.einsum` function, which implements the so-called [Einstein summation convention (link to Wiki)](https://en.wikipedia.org/wiki/Einstein_notation):

In [38]:
def _dist(A, B):
    """Compute the euclidean distance two matrices"""
    return np.sqrt(np.einsum("ijk->ij", (A[:, None, :] - B) ** 2))

In [39]:
V

array([[ 3.4642421 ,  9.24276663, 15.82553729],
       [ 2.74040953, 10.27474641, 15.98930677]])

In [40]:
X

array([[ 0.72852489, 11.94667841, 13.59914048],
       [ 8.79118537, 11.08317434, 14.81793144],
       [ 2.38801479,  4.14141081, 17.36653303],
       [ 4.2224803 , 17.04222896, 11.2299956 ],
       [ 4.77397008, 12.90549072, 16.72757254],
       [ 2.77734604,  7.72183846, 19.19693698],
       [ 0.89662189, 12.70659482, 18.96689418]])

In [41]:
_dist(X, V)**2

array([[19.75213031, 12.55593133],
       [32.77869472, 38.63756415],
       [29.55676429, 39.6387398 ],
       [82.52554128, 70.64639669],
       [15.94460273, 11.60122044],
       [14.1513846 , 16.80759488],
       [28.45890242, 18.17946634]])

In [42]:
(np.round(_dist(X, V)**2, 4) != np.round(D, 4)).sum()

0

---

We now need to calculate matrix $E$ out of $D$.

In [43]:
D

array([[19.75213031, 12.55593133],
       [32.77869472, 38.63756415],
       [29.55676429, 39.6387398 ],
       [82.52554128, 70.64639669],
       [15.94460273, 11.60122044],
       [14.1513846 , 16.80759488],
       [28.45890242, 18.17946634]])

In [44]:
D.shape

(7, 2)

Firstly, we introduce a new artificial axis, to work with a 3-dimensional array. We can do it in two ways: by `D[:, np.newaxis, :]` or by `D.reshape(n_samples, 1, -1)`:

In [45]:
D_reshaped = D.reshape(n_samples, 1, -1)

In [46]:
D_reshaped.shape

(7, 1, 2)

In [47]:
D_reshaped

array([[[19.75213031, 12.55593133]],

       [[32.77869472, 38.63756415]],

       [[29.55676429, 39.6387398 ]],

       [[82.52554128, 70.64639669]],

       [[15.94460273, 11.60122044]],

       [[14.1513846 , 16.80759488]],

       [[28.45890242, 18.17946634]]])

In [48]:
sum(D[:, np.newaxis, :] != D_reshaped)

array([[0, 0]])

We now repeat each individual entry 1 time over the last axis. We effectively create a duplicated row containing distances of a given observation to all cluster prototypes.

In [49]:
D_repeated = D_reshaped.repeat(D.shape[-1], axis=1)

In [50]:
D_repeated.shape

(7, 2, 2)

In [51]:
D_repeated

array([[[19.75213031, 12.55593133],
        [19.75213031, 12.55593133]],

       [[32.77869472, 38.63756415],
        [32.77869472, 38.63756415]],

       [[29.55676429, 39.6387398 ],
        [29.55676429, 39.6387398 ]],

       [[82.52554128, 70.64639669],
        [82.52554128, 70.64639669]],

       [[15.94460273, 11.60122044],
        [15.94460273, 11.60122044]],

       [[14.1513846 , 16.80759488],
        [14.1513846 , 16.80759488]],

       [[28.45890242, 18.17946634],
        [28.45890242, 18.17946634]]])

We introduce a new axis the same way to the original distances matrix $D$, but this time without repeating the entries:

In [52]:
D_newaxis = D[:, :, np.newaxis]

In [53]:
D_newaxis.shape, D_repeated.shape

((7, 2, 1), (7, 2, 2))

In [54]:
D_newaxis

array([[[19.75213031],
        [12.55593133]],

       [[32.77869472],
        [38.63756415]],

       [[29.55676429],
        [39.6387398 ]],

       [[82.52554128],
        [70.64639669]],

       [[15.94460273],
        [11.60122044]],

       [[14.1513846 ],
        [16.80759488]],

       [[28.45890242],
        [18.17946634]]])

We now divide `D_newaxis` by `D_repeated`. By the mechanism of broadcasting, the final entry in our 3-dimensional matrix will be a row containing

$$
\frac{d^2(x_j, v_k)}{d^2(x_j, v_1)},
\frac{d^2(x_j, v_k)}{d^2(x_j, v_2)},
\ldots,
\frac{d^2(x_j, v_k)}{d^2(x_j, v_c)}
$$

for each of the clusters.

In [55]:
D_newaxis / D_reshaped

array([[[1.        , 1.57313144],
        [0.63567479, 1.        ]],

       [[1.        , 0.84836339],
        [1.17874017, 1.        ]],

       [[1.        , 0.74565348],
        [1.34110552, 1.        ]],

       [[1.        , 1.16814933],
        [0.85605493, 1.        ]],

       [[1.        , 1.37439012],
        [0.72759545, 1.        ]],

       [[1.        , 0.84196369],
        [1.18769967, 1.        ]],

       [[1.        , 1.56544213],
        [0.63879717, 1.        ]]])

Indeed, lets examine a first sub-matrix:

In [56]:
(D_newaxis / D_reshaped)[0, :, :]

array([[1.        , 1.57313144],
       [0.63567479, 1.        ]])

Recall entries from $D$

In [57]:
D[0, :]

array([19.75213031, 12.55593133])

`(D_newaxis / D_reshaped)[0, :, :]`  is a matrix

$$
\begin{bmatrix}
\frac{d^2(x_1, v_1)}{d^2(x_1, v_1)} &
\frac{d^2(x_1, v_1)}{d^2(x_1, v_2)} \\
\frac{d^2(x_1, v_2)}{d^2(x_1, v_1)} &
\frac{d^2(x_1, v_2)}{d^2(x_1, v_2)}
\end{bmatrix}
$$

We can now sum over the 2-nd axis

In [58]:
E_reciprocal = (D_newaxis / D_reshaped).sum(axis=2)

In [59]:
E_reciprocal

array([[2.57313144, 1.63567479],
       [1.84836339, 2.17874017],
       [1.74565348, 2.34110552],
       [2.16814933, 1.85605493],
       [2.37439012, 1.72759545],
       [1.84196369, 2.18769967],
       [2.56544213, 1.63879717]])

Each $j-$th row of `E_reciprocal` is a $2-$tuple

$$
\bigg( \sum\limits_{s=1}^2 \frac{d^2(x_j, v_1)}{d^2(x_j, v_s)}, \sum\limits_{s=1}^2 \frac{d^2(x_j, v_2)}{d^2(x_j, v_s)} \bigg)
$$

Recall that the data evidence is a reciprocal of     $\sum_{s=1}^{c}
    \frac{
        d^2(x_j, v_k)
    }{
       d^2(x_j, v_s)
    }$, so

In [60]:
E = 1 / E_reciprocal

In [61]:
E

array([[0.38863153, 0.61136847],
       [0.54101916, 0.45898084],
       [0.57285138, 0.42714862],
       [0.46122284, 0.53877716],
       [0.42116078, 0.57883922],
       [0.54289887, 0.45710113],
       [0.38979636, 0.61020364]])

## Multiplier matrix $M$

Further on, the evidence value $e_{jk}$ is modified for both $u_{i,k \neq s(i)}$ and $u_{i, s(i)}$ by a factor of $\frac{1}{1+\alpha}$.

In a matrix form, we express it by matrix $M$ that is built out of $F$:

$$
F=
\begin{bmatrix}
1 & 0 \\
1 & 0 \\
0 & 1 \\
0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\
\end{bmatrix}
$$

$$
M=
\begin{bmatrix}
\frac{1}{1+\alpha} & \frac{1}{1+\alpha} \\
\frac{1}{1+\alpha} & \frac{1}{1+\alpha} \\
\frac{1}{1+\alpha} & \frac{1}{1+\alpha} \\
1 & 1 \\ 1 & 1 \\ 1 & 1 \\ 1 & 1 \\
\end{bmatrix}
$$

A first step to build a matrix formula for $U$ is to multiply element-wise $M$ and $E$, arriving at

$$
M \cdot E = ME = [me_{jk}] \text{ , where } me_{jk} = \frac{1+\alpha \cdot (1-b_j \sum_{s=1}^c f_{js})}{1+\alpha} 
\cdot
\bigg(
\sum_{s=1}^{c}
    \frac{
        d^2(x_j, v_k)
    }{
       d^2(x_j, v_s)
    }
\bigg)^{-1}
$$

where $A \cdot B$ is an element-wise operation of multiplication of elements of matrices $A$ and $B$.

### `Python` code

Recall we already have information about indices of supervised observations in matrix $F$

In [62]:
i_indices

array([0, 1, 2])

In [63]:
F

array([[1, 0],
       [1, 0],
       [0, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])

In [64]:
M = np.ones(F.shape)

In [65]:
M[i_indices, :] = 1 / (1 + alpha)

In [66]:
M

array([[0.55555556, 0.55555556],
       [0.55555556, 0.55555556],
       [0.55555556, 0.55555556],
       [1.        , 1.        ],
       [1.        , 1.        ],
       [1.        , 1.        ],
       [1.        , 1.        ]])

## Absolute Lower Bound matrix $ALB$

Finally, the true power of the scaling factor $\alpha$ in SSFCM is the Absolute Lower Bound operation.
For each $u_{i,s(i)}$ a value $\frac{\alpha}{1+\alpha}$ is added. In a matrix form, it can be expressed as

$$
F=
\begin{bmatrix}
1 & 0 \\
1 & 0 \\
0 & 1 \\
0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\
\end{bmatrix}
$$

$$
ALB=
\begin{bmatrix}
\frac{\alpha}{1+\alpha} & 0 \\
\frac{\alpha}{1+\alpha} & 0 \\
0 & \frac{\alpha}{1+\alpha} \\
0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\
\end{bmatrix}
$$

### `Python` code

In [67]:
ALB = F * ( alpha / (1 + alpha) )

## Final formula for $U$

$U = ME + ALB$

In [68]:
U = np.multiply(M, E) + ALB

In [69]:
U

array([[0.66035085, 0.33964915],
       [0.74501064, 0.25498936],
       [0.31825076, 0.68174924],
       [0.46122284, 0.53877716],
       [0.42116078, 0.57883922],
       [0.54289887, 0.45710113],
       [0.38979636, 0.61020364]])

Let's compare memberships $u_{i,s(i)}$ stemming from (a) $U^\text{SSFCM}$ matrix in SSFCM, and (b) $U^\text{FCM}$ matrix in regular FCM.

(a) $U^\text{SSFCM}$ = `U`,<br>
(b) $U^\text{FCM}$ = `E`

In [70]:
U

array([[0.66035085, 0.33964915],
       [0.74501064, 0.25498936],
       [0.31825076, 0.68174924],
       [0.46122284, 0.53877716],
       [0.42116078, 0.57883922],
       [0.54289887, 0.45710113],
       [0.38979636, 0.61020364]])

In [71]:
F

array([[1, 0],
       [1, 0],
       [0, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])

supervised memberships in U

In [72]:
U[F.nonzero()]

array([0.66035085, 0.74501064, 0.68174924])

supervised memberships in E

In [73]:
E[F.nonzero()]

array([0.38863153, 0.54101916, 0.42714862])

*This is an example of indexing specific elements from 2-d arrays taken from [numpy docs](https://numpy.org/devdocs/user/basics.indexing.html)

```python
y = np.arange(35).reshape(5, 7)

y
array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13],
       [14, 15, 16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25, 26, 27],
       [28, 29, 30, 31, 32, 33, 34]])

y[np.array([0, 2, 4]), np.array([0, 1, 2])]
array([ 0, 15, 30])
```

Note that `matrix.nonzero()` provides a 2-tuple of such arrays required to select elements!

### End of step $l=1$: checking convergence

In [74]:
U

array([[0.66035085, 0.33964915],
       [0.74501064, 0.25498936],
       [0.31825076, 0.68174924],
       [0.46122284, 0.53877716],
       [0.42116078, 0.57883922],
       [0.54289887, 0.45710113],
       [0.38979636, 0.61020364]])

In [75]:
U_dummy

array([[0.3197939 , 0.6802061 ],
       [0.63584491, 0.36415509],
       [0.80844318, 0.19155682],
       [0.58547838, 0.41452162],
       [0.41186536, 0.58813464],
       [0.392291  , 0.607709  ],
       [0.45717935, 0.54282065]])

In [76]:
error = 1e-4

In [77]:
if np.linalg.norm(U - U_dummy) < error:
    print("Stop! Convergence reached.")
else:
    print("Need to continue. Increase $l$ by 1")

Need to continue. Increase $l$ by 1
